In [1]:
! pip -q install -e ..

In [2]:
import weboftruth as wot

In [3]:
from os.path import join

from torch import cuda
from torch.optim import Adam 
import torch

import pandas as pd
import torchkge
from torchkge.models import TransEModel
from torchkge.sampling import BernoulliNegativeSampler
from torchkge.utils import MarginLoss, DataLoader
# from torchkge.utils.datasets import load_fb15k237
from torchkge.utils.datasets import load_wikidatasets
from torchkge.data_structures import KnowledgeGraph

import argparse
import os

path = '..'
wot.svo_data_path = join(path, 'data', 'SVO-tensor-dataset')
wot.svo_paths = {k:join(wot.svo_data_path, str(k)) for k in [100, 80, 50]}
wot.models_path = join(path, 'models')
#print(wot.svo_paths)
#wot.reset_paths('..')
#print(wot.svo_paths)

def modelslist(module):
    return [x for x in dir(module) if 'model' in x.lower()]

def exp_obj(x):
    print([_ for _ in dir(x) if '__' not in _])

In [4]:
df = pd.DataFrame({'from': {0: 'a', 1: 'b', 2: 'c'},
 'to': {0: 'b', 1: 'c', 2: 'b'},
 'rel': {0: '1', 1: '2', 2: '3'}})
kg = KnowledgeGraph(df)

In [5]:
kg.relations

tensor([0, 1, 2])

In [6]:
[k for k in dir(kg) if '__' not in k]

['dict_of_heads',
 'dict_of_tails',
 'ent2ix',
 'evaluate_dicts',
 'get_mask',
 'get_sizes',
 'head_idx',
 'n_ent',
 'n_facts',
 'n_rel',
 'rel2ix',
 'relations',
 'sanity_check',
 'split_kg',
 'tail_idx']

In [7]:
kg.dict_of_heads

defaultdict(set, {(1, 0): {0}, (2, 1): {1}, (1, 2): {2}})

In [8]:
kg.rel2ix

{'1': 0, '2': 1, '3': 2}

In [9]:
kg.ent2ix

{'a': 0, 'b': 1, 'c': 2}

In [10]:
path = '/Users/aabir/Documents/research/weboftruth'
model_type = 'TransE'
epochs = 5
small = True
ts = 100

print(f"Path: {path}\nModel Type: {model_type}")
print(f"Epochs: {epochs}\nSmall: {small}")
print(f"Truth share: {ts}")
for f in os.listdir(wot.svo_paths[ts]):
    if 'train' in f: tr_fn = f
    if 'valid' in f: val_fn = f
    if 'test' in f: test_fn = f


Path: /Users/aabir/Documents/research/weboftruth
Model Type: TransE
Epochs: 5
Small: True
Truth share: 100


In [11]:

#dfs = wot.utils.read_data(tr_fn, val_fn, test_fn,
#                            wot.svo_paths[ts])
#dfs = [df.drop('true_positive', axis=1
#            ) if 'true_positive' in df.columns else df
#            for df in dfs ]
#tr_kg, val_kg, test_kg = (wot.utils.df_to_kg(df) for df in dfs)

tr_kg = kg

mod = wot.CustomTransModel(tr_kg, model_type=model_type,
                                ts=ts, ent_emb_dim=2, rel_emb_dim=2)

!! 3 3
Creating TransE in folder: ../models/TransE_05


In [12]:

mod.set_sampler(samplerClass=BernoulliNegativeSampler, kg=tr_kg)
mod.set_optimizer(optClass=Adam)
mod.set_loss(lossClass=MarginLoss, margin=0.5)
# Move everything to CUDA if available
if cuda.is_available():
    print("Using cuda.")
    cuda.empty_cache()
    cuda.init()
    mod.model.cuda()
    mod.loss_fn.cuda()
mod.train_model(10, tr_kg)
mod.validate(tr_kg)


 saving fTransE to ../models/TransE_05/e=1_ts=_TransE_model.pt
 saving fTransE to ../models/TransE_05/best_e=1_ts=_TransE_model.pt
 saving fTransE to ../models/TransE_05/e=10_ts=_TransE_model.pt



3.6832730770111084

In [13]:
type(mod)

weboftruth.wotmodels.CustomTransModel

In [17]:
mod.trainkg

In [18]:
mod.model

TransEModel(
  (ent_emb): Embedding(3, 250)
  (rel_emb): Embedding(3, 250)
)

In [19]:
ents, rels = mod.model.get_embeddings()

In [22]:
mod.trainkg.ent2ix

{'a': 0, 'b': 1, 'c': 2}

In [24]:
kg.relations, kg.ent2ix

(tensor([0, 1, 2]), {'a': 0, 'b': 1, 'c': 2})

In [ ]:
for hr, t in [(k, v) for k, (v,) in kg.dict_of_tails.items()]:
    print(hr[0], hr[1], t)

In [26]:
wot.utils.kg_to_df(kg)

,from,to,rel
0,a,b,1
1,b,c,2
2,c,b,3
